# Interactive White Matter Segmentation

In this notebook you'll have the opportunity to perform an interactive, anatomically based white matter segmentation right here in the browser, using your own data.  This will allow you to explore the anatomical connectivity instantiated in your tractogram to get a sense of the quality of your tractography data, what parts of the brain are connected to what (according to your tracrogram), and how well your volumetric atlas serves your needs.  

## Initial setup
Before we can do any of this though, we have to make sure this notebook is setup correctly and can access your data.

### Using this notebook locally

If you're using this notebook locally, one of the foremost things you should ensure is that your local python environment has the necessary packages installed.  Check the **requirements.txt** file in this repository to determine which packages are necessary.

When running this notebook locally, you should be able to navigate to the appropriate folder(s) containing your data using the file chooser provided by the subsequent cells.

### Jupyterlab / binder

If you're using a [jupyterlab](https://jupyter.org/) [binder](https://mybinder.org/), there's an extra step to making your data avaialble to this interface.  In order to do this, you'll need to upload your data to this virtual environment.  NOTE: because this is a temporary virtual environment, any data that you upload here will not be preserved.  All the same, it is best to be careful about what data is uploaded.  To upload data to this [jupyterlab](https://jupyter.org/) environment Click the **file icon** in the **left hand menu**, then click the **underlined up arrow** to begin the upload selection.  This should be sufficient setup, as package managment is taken care of by these platforms.

### Jupyter Notebook Classic / binder

Alternatively, if you're using a classic jupyter-notebook [binder](https://mybinder.org/) (acheived by clicking **Launch Classic Notebook** under the **help menu**, and temporarily _reccomended_ until widget interfacing has been fixed in jupyterlab) click the **upload** button towards the upper right hand side of the interface and upload the files below, and then select this notebook (InteractiveWhiteMatterSegmentation.ipynb) and proceed normally.

## Target data

For this particular notebook to operate successfully you'll need to upload the following:

- The desired volumetric brain segmentation that will be used to specify brain areas that you wish to view connectivity between (the atlas and the tractrogram should be aligned to one another / in the same space).
- A whole-brain tractogram containing the candidate streamlines you wish to inspect/segment (the atlas and the tractrogram should be aligned to one another / in the same space).
- A lookup table (csv or xls) indicating the human-interpretable identities of the labels in the volumetric brain atlas file.

In the several of the following cells, you will be asked to select/specify your data targets.  Their location will depend on the method by which you are interfacing with this notebook (as described in the [Initial setup](#Initial setup) section.  As with any jupyter notebook, you will need to hit **run** for each cell in order to proceed.  For data upload blocks this will trigger the provision of a file-chooser interface.  For other code blocks, running the cell will result in the production of a visualization, table, or other graphical/interactive feature.

## The Atlas

### Selecting the atlas
We'll begin by selecting the volumetric brain atlas that you would like to use to select your streamlines of interest.  Later on, you'll be selecting labels/regions from this atlas as a means of indicating that you would like to visualize streamlines that begin or end (tractography is agnostic with respect to directionality) in these areas.

In [ ]:
#upload widget not working right now, just use path specification for current purposes
#import ipywidgets as widgets
#uploader = widgets.FileUpload()
#uploader

#set path to atlas
from ipyfilechooser import FileChooser
import os

# Create and display a FileChooser widget
# if a file has already been selected, do not revert to no file selected
if 'fcAtlas' in locals():
    #however, if the selected file path is None, do rectify that
    if fcAtlas.selected_path==None:
        print('Please select your input atlas file')
        fcAtlas = FileChooser(os.getcwd())
        display(fcAtlas)
    else:
        # just redisplay what you already have
        fcAtlas.default_path = fcAtlas.selected_path
        fcAtlas.default_filename=fcAtlas.selected_filename
        display(fcAtlas)
#if it doesn't exist at all (not sure how that would happen) also be sure to rectify that
elif not('fcAtlas' in locals()):
    fcAtlas = FileChooser(os.getcwd())
    print('Please select your input atlas file')
    display(fcAtlas)

## The reference anatomy

In order to get a better look at what has been segmented (and because there is no reference anatomy in the above visualization) we can also convert these selected streamlines to a density map and plot them on a reference T1.  To do this we'll need to upload a reference T1 in the same fashion as our other data objects.

**Note**: this will only be useful _if_ the T1 image is aligned to / in the same space as the tractography.

(after selecting the file to upload, run the subsequent cell as well in order to create an interactive visualizer)

In [ ]:
# Create and display a FileChooser widget
# if a file has already been selected, do not revert to no file selected
from ipyfilechooser import FileChooser
import os

if 'T1path' in locals():
    #however, if the selected file path is None, do rectify that
    if T1path.selected_path==None:
        print('Please select your T1 file')
        T1path = FileChooser(os.getcwd())
        display(T1path)
    else:
        # just redisplay what you already have
        T1path.default_path = T1path.selected_path
        T1path.default_filename=T1path.selected_filename
        display(T1path)
#if it doesn't exist at all (not sure how that would happen) also be sure to rectify that
elif not('T1path' in locals()):
    T1path = FileChooser(os.getcwd())
    print('Please select your input T1 file')
    display(T1path)

### Loading and inspecting the atlas

Now that we have selected the atlas file, let's take a moment to inspect it.  Run the next cell, and then feel free to use the X, Y, and Z sliders to move about the atlas volume.  Although it is not possible from this visualization to interpret which labels are where, it is possible to get a sense of what the atlas's coverage is.

In [ ]:
#conditionally plot the atlas if a file has been chosen.  If not, print warning.
#this might be avoidable with a clever use of observe

import copy
import numpy as np
from dipy.tracking.utils import reduce_labels
if not(fcAtlas.selected_path==None):
    pathToAtlas=os.path.join(fcAtlas.selected_path,fcAtlas.selected_filename)
    #load the nifti
    import nibabel as nib
    atlasImg = nib.load(pathToAtlas)
    T1ToLoad=(os.path.join(T1path.selected_path,T1path.selected_filename))
    t1img = nib.load(T1ToLoad)
    from nilearn import plotting

    print('relabeling atlas')
    #zero index necessary because function returns tuple?
    relabeledAtlasData=reduce_labels(atlasImg.get_fdata())[0]

    #save to working directory; required as nifitWidget doesn't load from object apparently
    renumberedAtlasNifti=nib.Nifti1Image(relabeledAtlasData, atlasImg.affine, atlasImg.header) 
    #save the object down
    nib.save(renumberedAtlasNifti, 'relabeledAtlas.nii.gz')

    #create a distinguishable colormap
    import matplotlib.pyplot as plt
    uniqueColors=plt.cm.get_cmap('hsv', np.max(renumberedAtlasNifti.get_fdata()))
    
    display(plotting.view_img(stat_map_img=renumberedAtlasNifti, bg_img=t1img,dim=2 ,opacity=.5, cut_coords=None,colorbar=True,cmap=uniqueColors, symmetric_cmap=False).resize(900,400))
else:
    print('No input atlas file selected.  Please return to previous cell, select an atlas file, and rerun this cell.')

## The lookup table

### Selecting the lookup table

Next we'll select the Lookup table for the atlas you have provided.  The Lookup table provides information about the correspondances of the numerical labels in the atlas file you just provided with the anatomical/conceptual entities they are supposed to designate (as described [here, in a broad manner](https://dannbullock.github.io/WiMSE/notebooks/How_to_interpret_a_volumetric_brain_segmentation.html) and [here, in the context of color lookup tables](https://www.slicer.org/wiki/Documentation/4.1/SlicerApplication/LookupTables).

We will be using an interactive interface with a spreadsheet version of the lookup table in order to select which labels will be required endpoints for our streamlines of interest.  Technically you do not need to provide an Lookup table, as this notebook will peice together a spreadsheet label-selection interface in one is not found, however such an interface will have no terminology-based labeling information and will thus be fairly difficult to interpret or work with. 

In [ ]:
#set path to atlas
from ipyfilechooser import FileChooser
import os

if 'fcLUT' in locals():
    #however, if the selected file path is None, do rectify that
    if fcLUT.selected_path==None:
        print('Please select your input Lookup table file')
        fcLUT = FileChooser(os.getcwd())
        display(fcLUT)
    else:
        # just redisplay what you already have
        fcLUT.default_path = fcLUT.selected_path
        fcLUT.default_filename=fcLUT.selected_filename
        display(fcLUT)
#if it doesn't exist at all (not sure how that would happen) also be sure to rectify that
elif not('fcLUT' in locals()):
    fcLUT = FileChooser(os.getcwd())
    print('Please select your input Lookup table file')
    display(fcLUT)

### Inspecting and interacting with the lookup table

Lets take a quick look at the LUT that has been provided (or that will be built) using [qgrid](https://github.com/quantopian/qgrid).

In [ ]:
#conditionally plot the atlas if a file has been chosen.  If not, build a table from the information in the passed atlas.
#this might be avoidable with a clever use of observe
import pandas as pd
if not(fcLUT.selected_path==None):
    LUTpath=os.path.join(fcLUT.selected_path,fcLUT.selected_filename)
    if fcLUT.selected_filename[-4:]=='.csv':
        LUT=pd.read_csv(LUTpath)
    elif (fcLUT.selected_filename[-4:]=='.xls',fcLUT.selected_filename[-5:]=='.xlsx'):
        LUT=pd.read_excel(LUTpath)
else:
    #build it from the input data
    import numpy as np
    inferredLUTIdentities={'origLabels': np.unique(atlasImg.get_fdata()).astype(int),'newLabels':np.unique(renumberedAtlasNifti.get_fdata()).astype(int)}
    LUT=pd.DataFrame(data=inferredLUTIdentities)  

#one way or the other the LUT should now be available
import qgrid
qgrid_firstLUT_widget= qgrid.show_grid(LUT,show_toolbar=True)
qgrid_firstLUT_widget

## The tractogram

### Selecting the tractogram 

Next, let's select the tractogram file.  Due to the [RAM limitations of binder](https://mybinder.readthedocs.io/en/latest/about/about.html#how-much-memory-am-i-given-when-using-binder), if you are using binder or a similar service, it is **strongly** recomend that you limit the size of the uploaded tractogram to less than 1.25 GB.  Otherwise, tractogram limitations are associated with your local hardware (in particular RAM).

In [ ]:
# Create and display a FileChooser widget
# if a file has already been selected, do not revert to no file selected
if 'fcTractogram' in locals():
    #however, if the selected file path is None, do rectify that
    if fcTractogram.selected_path==None:
        print('Please select your input tractogram file')
        fcTractogram = FileChooser(os.getcwd())
        display(fcTractogram)
    else:
        # just redisplay what you already have
        fcTractogram.default_path = fcTractogram.selected_path
        fcTractogram.default_filename=fcTractogram.selected_filename
        display(fcTractogram)
#if it doesn't exist at all (not sure how that would happen) also be sure to rectify that
elif not('fcTractogram' in locals()):
    fcTractogram = FileChooser(os.getcwd())
    print('Please select your input tractogram file')
    display(fcTractogram)

## Selecting your rois

Next, we'll use the refernce T1 image to help us place a spherical ROI, which we will then use to segment streamlines from our tractogram.

In [ ]:
#this code ensures that we can navigate the WiMSE repo across multiple systems
import subprocess
import os
#get top directory path of the current git repository, under the presumption that 
#the notebook was launched from within the repo directory
gitRepoPath=subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode('ascii').strip()

#establish path to the wma_tools repo
wma_toolsDirPath=os.path.join(gitRepoPath,'wma_pyTools')   

#change to the wma_tools path, load the function set, then change back to the top directory
os.chdir(wma_toolsDirPath)
import WMA_pyFuncs
os.chdir(gitRepoPath)

import pandas as pd
import qgrid
import numpy as np
segmentationQueueDF=pd.DataFrame(columns=['ROI_type','centerPoint','dimension or radius','include'])
#we'll go ahead and populate the first record with a 5 mm sphere in the middle of the volume
convertedBoundCoords=WMA_pyFuncs.subjectSpaceMaskBoundaryCoords(t1img)
#this is the example query
exampleQueryRow=['sphere',np.mean(convertedBoundCoords,axis=0),5,True]
#convert it to a series
querySeries = pd.Series(exampleQueryRow, index = segmentationQueueDF.columns)
#append it
segmentationQueueDF = segmentationQueueDF.append(querySeries, ignore_index=True)
#render it into an interactive widget
qgrid_segmentationQueue_widget= qgrid.show_grid(segmentationQueueDF,show_toolbar=True)
qgrid_segmentationQueue_widget

In [ ]:
from distutils import util

roisNifti =[]
include = []
operations= []
#get the resultant dataframe
from dipy.tracking.utils import reduce_rois
segmentationQueueDF=qgrid_segmentationQueue_widget.get_changed_df()

    # generate some number of spheres to use in a test segmentation
for iRois in list(range(len(segmentationQueueDF.index))):
    #if it requests a sphere, create a sphere
    if segmentationQueueDF.at[iRois,'ROI_type'].lower()=='sphere':
        # obtain that data array as bool
        currentROI=WMA_pyFuncs.createSphere(segmentationQueueDF.at[iRois,'dimension or radius'], segmentationQueueDF.at[iRois,'centerPoint'], t1img)
    elif segmentationQueueDF.at[iRois,'ROI_type'].lower()=='plane':
        # add that and a True to the list vector for each
        currentROI=WMA_pyFuncs.makePlanarROI(t1img, float(segmentationQueueDF.at[iRois,'centerPoint']), segmentationQueueDF.at[iRois,'dimension or radius'])
    else:
        raise Exception("Input ROI type specification not understood.")
        
    #in either case
    roisNifti.append(currentROI)
    #just seting this to any, can make this an option later
    operations.append('any')
    # convert include vector robustly
    include.append(bool(util.strtobool(str(segmentationQueueDF.at[iRois,'include']))))
    
#obtain the data from each roi
roisData=[iROI.get_fdata().astype(bool) for iROI in roisNifti]
#condense the include and exclude ROIS
include_roi, exclude_roi=reduce_rois(roisData, include)
#create blank data object to serve as the merge,really only for visualization purposes
roisMergeData=np.zeros(include_roi.shape, dtype=int)
#set the entries to their respective values
roisMergeData[include_roi]=1
roisMergeData[exclude_roi]=-1
mergedRoisNifti=nib.Nifti1Image(roisMergeData, t1img.affine, t1img.header) 
#plot these, for inspection
display(plotting.view_img(stat_map_img=mergedRoisNifti, bg_img=t1img,dim=2 ,opacity=.5, cut_coords=None,colorbar=True,cmap='jet').resize(900,400))

## The segmentation

### Performing the segmentation

Now that we've entered the rois and visualized them, we can perform the segmentation


In [ ]:
#load up the tractogram if possible
if not(fcTractogram.selected_path==None):   
    tractogramPath=(os.path.join(fcTractogram.selected_path,fcTractogram.selected_filename))
    #do a check to see if the selected input file type is supported
    if not(nib.streamlines.is_supported(tractogramPath)):
        raise ValueError("Input tractogram file type not supported")
    else:
        streamsObjIN=nib.streamlines.load(tractogramPath)

def extractSubTractogram(sourceTractogram,indexes):
    #import relevant package
    import nibabel as nib
    #extract the desired streamlines into a new streamline object
    streamlines = sourceTractogram.streamlines[indexes]
    #establish tractogram object
    out_tractogram = nib.streamlines.tractogram.Tractogram(streamlines)
    return out_tractogram

#interactive plotting via niwidgets?  
#widget within a widget doesn't seem to work
def plotSegmentedStreamsWidget(subTractogram):
    #import widget
    from niwidgets import StreamlineWidget
    #set widget object
    sw = StreamlineWidget(streamlines=subTractogram)
    #set plotting characteristics
    style = {'axes': {'color': 'red',
                  'label': {'color': 'white'},
                  'ticklabel': {'color': 'white'},
                  'visible': True},
         'background-color': 'black',
         'box': {'visible': True}}
    #plot it
    %matplotlib inline
    display(sw.plot(display_fraction=1, width=1000, height=1000, style=style, percentile=0))
    
import numpy as np
import matplotlib.pyplot as plt  

#use the criteria application function to find the relevant streamlines
streamBool=WMA_pyFuncs.segmentTractMultiROI(streamsObjIN.streamlines, roisNifti, include , operations)
    
streamsToPlot=extractSubTractogram(streamsObjIN,np.where(streamBool)[0])
  
display(plotSegmentedStreamsWidget(streamsToPlot.streamlines))

### Examining streamline density relative to anatomy

In order to obtain a more informed understanding of the selected streamlines and their location in the brain, we can plot their density overlain on the T1 anatomy as a heatmap.

In [ ]:
#load selected t1 image
import nibabel as nib
import os
#load the T1
T1ToLoad=(os.path.join(T1path.selected_path,T1path.selected_filename))
t1img = nib.load(T1ToLoad)

#use dipy to create the density mask
from dipy.tracking import utils
densityMap=utils.density_map(streamsToPlot.streamlines, t1img.affine, t1img.shape)
densityNifti=nib.nifti1.Nifti1Image(densityMap,t1img.affine, t1img.header)

from nilearn import plotting
plotting.view_img(stat_map_img=densityNifti, bg_img=t1img, cut_coords=None,colorbar=True,vmin=0,vmax=np.max(densityNifti.get_fdata()),cmap='viridis', symmetric_cmap=False).resize(900,400)

### Where are these streamlines terminating

We may also be curious about what areas from the input atlas these streamlines are terminating in.  To this end we can perform a quick atlas based segmentation and plot the quantative results 

In [ ]:
from dipy.tracking import utils
#segment tractome into connectivity matrix from parcellation
M, grouping=utils.connectivity_matrix(streamsToPlot.streamlines, atlasImg.affine, label_volume=renumberedAtlasNifti.get_fdata().astype(int),
                            return_mapping=True,
                            mapping_as_streamlines=False)
# if the length of the provided LUT exceeds the number of unique entries, then the indexes of the table cannot be relied
#upon to provide us with the correspondance to the relabeled atlas.  If this is the case, try and implement a workaround.
if len(LUT)>len(np.unique(relabeledAtlasData)):
    #infer which column contains the original identities
    #presumably, this would be the LUT column with the largest number of matching labels with the original atlas.
    matchingLabelsCount=[len(list(set(LUT[iColumns]).intersection(set(np.unique(atlasImg.get_fdata()).astype(int))))) for iColumns in LUT.columns.to_list()]
    #there's an edge case here relabeled atlas == the original atlas AND the provided LUT was larger (what would the extra entries be?)
    #worry about that later
    columnBestGuess=LUT.columns.to_list()[matchingLabelsCount.index(np.max(matchingLabelsCount))]
    #now that we have the guess, get the corresponding row entries, and reset the index.
    #This should make the index match the renumbered label values.
    LUTinterface=LUT[LUT[columnBestGuess].isin(np.unique(atlasImg.get_fdata()).astype(int)).values].reset_index(drop=True)

#in order to provide some intuitons about where to look, add a column indicating the number of endpoints in that label.
#this could probably just be an interative sum across row operation for the connectivty matrix.
connectionLocations=[]
#create a list of lists of all if the keys featuring this index
for iIndexes in LUT.index.to_list():
    connectionLocations.append( [iIndexes in sublist for sublist in list(grouping.keys())])
    
#get the dictionary keys in a list
keyTargets=list(grouping.keys())
#create a blank output vector
streamCounts=np.zeros(len(LUTinterface),dtype=int)
#iterate across all labels
for iLabels in LUTinterface.index.to_list():
    #find the dictionary keys which feature this connection
    currentKeylocations=[i for i, x in enumerate(connectionLocations[iLabels]) if x]
    #create a zero holder for a streamline accumulator/counter
    currentStreamTotal=0
    #iterate across the found connections for this key
    for iCurrentConnections in currentKeylocations:
        #add the number of streamlines to the streamline total
        currentStreamTotal=currentStreamTotal+len(grouping[keyTargets[iCurrentConnections]])
    #set the value
    streamCounts[iLabels]=currentStreamTotal
if not("endpointCount" in LUTinterface.columns):
    LUTinterface.insert(2, "endpointCount",streamCounts, True) 

import seaborn as sns
import matplotlib.pyplot as plt

# subset the labels to plot to only those that have streamlines
subsetDFtoPlot=LUTinterface[LUTinterface['endpointCount']>0]
# dynamically set plot dimensions
plotDim1=len(subsetDFtoPlot)*.3
plt.figure(figsize=(5,plotDim1))

#arbitrarily set for specific atlas for now, come back and make adaptive later
yColumn='Full_Name'
xColumn='endpointCount'
ax = sns.barplot(y=yColumn, x=xColumn, data=subsetDFtoPlot)
#set logscale
ax.set_xscale("log")
ax.set(title="Streamline connections found in current selection", xlabel='log scale streamline count', ylabel='atlas label')